<img src="https://images.efollett.com/htmlroot/images/templates/storeLogos/CA/864.gif" style="float: right;"> 




# ECON628-01 
### Lecture 1.2 - Classification with Logistic Regression and Evaluation Lab

## 1. Load affairs dataset
---
In this lab you will be using the data from the paper [A Theory of Extramarital Affairs] [1]. Here is the [link] [2] to the data dictionary
[1]: https://fairmodel.econ.yale.edu/rayfair/pdf/1978A200.PDF "A Theory of Extramarital Affairs"

[2]: https://vincentarelbundock.github.io/Rdatasets/doc/Ecdat/Fair.html "link"

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
affair_path = ('C:/Users/mjors_000/Desktop/ECON628-01-berenger92/datasets/affairs/affair.csv')
affair=pd.read_csv(affair_path)


In [4]:
print affair.head(2)

   Unnamed: 0     sex   age    ym child  religious  education  occupation  \
0           1    male  37.0  10.0    no          3         18           7   
1           2  female  27.0   4.0    no          4         14           6   

   rate  nbaffairs  
0     4          0  
1     4          0  


In [6]:
affair.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,601.0,301.000000,173.638033,1.000,151.0,301.0,451.0,601.0
age,601.0,32.487521,9.288762,17.500,27.0,32.0,37.0,57.0
ym,601.0,8.177696,5.571303,0.125,4.0,7.0,15.0,15.0
religious,601.0,3.116473,1.167509,1.000,2.0,3.0,4.0,5.0
education,601.0,16.166389,2.402555,9.000,14.0,16.0,18.0,20.0
occupation,601.0,4.194676,1.819443,1.000,3.0,5.0,6.0,7.0
rate,601.0,3.931780,1.103179,1.000,3.0,4.0,5.0,5.0
nbaffairs,601.0,1.455907,3.298758,0.000,0.0,0.0,0.0,12.0


In [8]:
affair.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 10 columns):
Unnamed: 0    601 non-null int64
sex           601 non-null object
age           601 non-null float64
ym            601 non-null float64
child         601 non-null object
religious     601 non-null int64
education     601 non-null int64
occupation    601 non-null int64
rate          601 non-null int64
nbaffairs     601 non-null int64
dtypes: float64(2), int64(6), object(2)
memory usage: 47.0+ KB


In [9]:
affair.corr()

,Unnamed: 0,age,ym,religious,education,occupation,rate,nbaffairs
Unnamed: 0,1.000000,0.036228,0.107761,-0.116447,-0.053710,-0.069130,-0.195132,0.576919
age,0.036228,1.000000,0.777546,0.193777,0.134596,0.166413,-0.199000,0.095237
ym,0.107761,0.777546,1.000000,0.218261,0.040003,0.044592,-0.243119,0.186842
religious,-0.116447,0.193777,0.218261,1.000000,-0.042571,-0.039722,0.024296,-0.144501
education,-0.053710,0.134596,0.040003,-0.042571,1.000000,0.533605,0.109303,-0.002437
occupation,-0.069130,0.166413,0.044592,-0.039722,0.533605,1.000000,0.017422,0.049612
rate,-0.195132,-0.199000,-0.243119,0.024296,0.109303,0.017422,1.000000,-0.279512
nbaffairs,0.576919,0.095237,0.186842,-0.144501,-0.002437,0.049612,-0.279512,1.000000


## 2. Encode nbaffairs as binary to create a 'had_affair' colum
---

We just want to see if people have had any affair or not.

In [12]:
def binary_affairs(x):
    if x==0:
        return 0
    else:
        return 1
#Creating another column that will use nbaffairs variable, and map it using the function above
affair['had_affair']= affair.nbaffairs.map(binary_affairs)

In [14]:
affair.head()

,Unnamed: 0,sex,age,ym,child,religious,education,occupation,rate,nbaffairs,had affair,had_affair
0,1,male,37.0,10.00,no,3,18,7,4,0,0,0
1,2,female,27.0,4.00,no,4,14,6,4,0,0,0
2,3,female,32.0,15.00,yes,1,12,1,4,0,0,0
3,4,male,57.0,15.00,yes,5,18,6,5,0,0,0
4,5,male,22.0,0.75,no,2,17,6,3,0,0,0


## 3. Create a dummy variables using pd.get_dummies
---
- affair_dumm = pd.get_dummies(affair)

http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.get_dummies.html

In [15]:
affair_dumm = pd.get_dummies(affair)
affair_dumm.head(4)

,Unnamed: 0,age,ym,religious,education,occupation,rate,nbaffairs,had affair,had_affair,sex_female,sex_male,child_no,child_yes
0,1,37.0,10.0,3,18,7,4,0,0,0,0,1,1,0
1,2,27.0,4.0,4,14,6,4,0,0,0,1,0,1,0
2,3,32.0,15.0,1,12,1,4,0,0,0,1,0,0,1
3,4,57.0,15.0,5,18,6,5,0,0,0,0,1,0,1


## 3. Setup X and Y matrices 
---

In [38]:
'''Define y and X'''
y = affair_dumm.had_affair.values
columns_ = affair_dumm.columns.tolist()
exclude_col = ['Unnamed: 0', 'had affair', 'had_affair', 'nbaffairs']

X = affair_dumm[[i for i in columns_ if i not in exclude_col]]

print (X.shape)
print (y.shape)

(601, 10)
(601L,)


## 4. Define your train and test data
---

In [39]:
'''Split the data'''
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=10)
print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape

(420, 10)
(420L,)
(181, 10)
(181L,)


## 5. Create and gridsearch a logistic regression, fit the regression, find best parameters, and baseline
---

In [40]:
'''Logistic function'''
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression()

'''set up parameters for the gridsearch I am using all the posible choices read documentation '''
## http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
Cs = np.logspace(0.0, 5.0, 20)

#WE ARE CREATING A DICTIONNARY THAT WILL FIND THE BEST MODEL FOR THE METHOD YOU CHOOSE (LASSO, RIDGE, ELAS..)
search_parameters = {
                    "penalty": ['l1','l2'],   
                    # Used to specify the norm used in the penalization.
                    
                    "C": Cs,  #Find the lambda value
                    # Regularization paramter
                    # "dual":[True, False], # Dual or primal formulation. Dual formulation is only implemented for l2 
                    # penalty with liblinear solver. Prefer dual=False when n_samples > n_features
                    
                    "fit_intercept": [False, True], 
                    # Specifies if a constant (a.k.a. bias or intercept) should be added to the decision function.
                    
                    "class_weight": [None, "balanced"], 
                    # The “balanced” mode uses the values of y to automatically adjust weights inversely 
                    # proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y))
                    
                    "intercept_scaling": [2, 1], 
                    # Useful only if solver is liblinear. when self.fit_intercept is True, instance vector x becomes 
                    # [x, self.intercept_scaling], i.e. a “synthetic” feature with constant value equals to intercept_scaling 
                    # is appended to the instance vector. 
                    
                    "solver":  ['liblinear']
 
}

'''Gridsearch your parameters''' #This is used to cross validate your findings in different sets of data

from sklearn.grid_search import GridSearchCV

## http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
estimator = GridSearchCV(logistic, search_parameters, cv=5, verbose=1, n_jobs=-1)

In [41]:
estimator.fit(X_train, y_train)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


[Parallel(n_jobs=-1)]: Done  60 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 1600 out of 1600 | elapsed:   14.1s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e+00,   1.83298e+00,   3.35982e+00,   6.15848e+00,
         1.12884e+01,   2.06914e+01,   3.79269e+01,   6.95193e+01,
         1.27427e+02,   2.33572e+02,   4.28133e+02,   7.84760e+02,
         1.43845e+03,   2.63665e+03,   4.83293e+03,   8.85867e+03,
         1.62378e+04,   2.97635e+04,   5.45559e+04,   1.00000e+05]), 'intercept_scaling': [2, 1], 'solver': ['liblinear'], 'fit_intercept': [False, True], 'penalty': ['l1', 'l2'], 'class_weight': [None, 'balanced']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [42]:
log_best = estimator.best_estimator_
print "Best estimators on the left out data:\n", log_best

print "\nBest C / Regularization Param on the left out data:\n", estimator.best_estimator_.C # This estimator.best_estimator_ object has many great reporting metrics
## Estimator that was chosen by the search, i.e. 
## estimator which gave highest score (or smallest loss if specified) on the left out data. 
## Not available if refit=False.

print "\nBest Params on hold out data (train):\n", estimator.best_params_
##Parameter setting that gave the best results on the hold out data.

print "\nBest Score on left out data:%.3f \n" % estimator.best_score_
## Score of best_estimator on the left out data.

Best estimators on the left out data:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Best C / Regularization Param on the left out data:
1.0

Best Params on hold out data (train):
{'C': 1.0, 'intercept_scaling': 1, 'fit_intercept': True, 'solver': 'liblinear', 'penalty': 'l1', 'class_weight': None}

Best Score on left out data:0.745 



In [43]:
'''Baseline'''
'''Remeber that 0 is republican 1 is democrat'''
print affair_dumm['had_affair'].value_counts(), "\n" 
print "if I randomly choose, %.0f percent of the time I will be choosing democrat " % ((np.mean(affair_dumm['had_affair']))*100)

0    451
1    150
Name: had_affair, dtype: int64 

if I randomly choose, 25 percent of the time I will be choosing democrat 


## 6. Evaluate your classfier (Confusion  Matrix,  Classification Report, ROC
---

- how many TP, TN, FP, FN values you have
- what is your accuracy, error rate, FPR, TPR, 

In [44]:
# Load Confusion Matrix package
from sklearn.metrics import confusion_matrix

In [45]:
'''This is the simplistic way to run a confusion Matrix'''
y_pred=estimator.predict(X_test)

confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
confmat

array([[138,   3],
       [ 32,   8]])

In [46]:
'''Pass it to a dataframe'''
confusion = pd.DataFrame(confmat, index=['True_Label_0 No affair', 'True_Label_1 Had affair'],
                         columns=['Predict_Label_0 No affair', 'Predict_Label_1 Had affair'])

confusion

,Predict_Label_0 No affair,Predict_Label_1 Had affair
True_Label_0 No affair,138,3
True_Label_1 Had affair,32,8


In [47]:
## Accuracy
## How often is the classifier correct?
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_pred)
print "Accuracy score: %.3f" %(acc*100)

Accuracy score: 80.663


In [ ]:
''' Code by Matt Hancock => thus far one of the best explanations of a Confusion Matrix
http://notmatthancock.github.io/2015/10/28/confusion-matrix.html'''

def show_confusion_matrix(C,class_labels=['0','1']):
    """
    C: ndarray, shape (2,2) as given by scikit-learn confusion_matrix function
    class_labels: list of strings, default simply labels 0 and 1.

    Draws confusion matrix with associated metrics.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    assert C.shape == (2,2), "Confusion matrix should be from binary classification only."
    
    # true negative, false positive, etc...
    tn = C[0,0]; fp = C[0,1]; fn = C[1,0]; tp = C[1,1];

    NP = fn+tp # Num positive examples
    NN = tn+fp # Num negative examples
    N  = NP+NN

    fig = plt.figure(figsize=(10,8))
    ax  = fig.add_subplot(111)
    ax.imshow(C, interpolation='nearest', cmap=plt.cm.gray)

    # Draw the grid boxes
    ax.set_xlim(-0.5,2.5)
    ax.set_ylim(2.5,-0.5)
    ax.plot([-0.5,2.5],[0.5,0.5], '-k', lw=2)
    ax.plot([-0.5,2.5],[1.5,1.5], '-k', lw=2)
    ax.plot([0.5,0.5],[-0.5,2.5], '-k', lw=2)
    ax.plot([1.5,1.5],[-0.5,2.5], '-k', lw=2)

    # Set xlabels
    ax.set_xlabel('Predicted Label', fontsize=16)
    ax.set_xticks([0,1,2])
    ax.set_xticklabels(class_labels + [''])
    ax.xaxis.set_label_position('top')
    ax.xaxis.tick_top()
    # These coordinate might require some tinkering. Ditto for y, below.
    ax.xaxis.set_label_coords(0.34,1.06)

    # Set ylabels
    ax.set_ylabel('True Label', fontsize=16, rotation=90)
    ax.set_yticklabels(class_labels + [''],rotation=90)
    ax.set_yticks([0,1,2])
    ax.yaxis.set_label_coords(-0.09,0.65)


    # Fill in initial metrics: tp, tn, etc...
    ax.text(0,0,
            'True Neg: %d\n(Num Neg: %d)'%(tn,NN),
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))

    ax.text(0,1,
            'False Neg: %d'%fn,
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))

    ax.text(1,0,
            'False Pos: %d'%fp,
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))


    ax.text(1,1,
            'True Pos: %d\n(Num Pos: %d)'%(tp,NP),
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))

    # Fill in secondary metrics: accuracy, true pos rate, etc...
    ax.text(2,0,
            'False Pos Rate: %.2f'%(fp / (fp+tn+0.)),
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))

    ax.text(2,1,
            'True Pos Rate: %.2f'%(tp / (tp+fn+0.)),
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))

    ax.text(2,2,
            'Accuracy: %.2f'%((tp+tn+0.)/N),
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))

    ax.text(0,2,
            'Neg Pre Val: %.2f'%(1-fn/(fn+tn+0.)),
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))

    ax.text(1,2,
            'Pos Pred Val: %.2f'%(tp/(tp+fp+0.)),
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))


    plt.tight_layout()
    plt.show()